In [67]:
# dependencies

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [8]:
benefits = pd.read_csv('benefits.csv')
exposure = pd.read_csv('exposure.csv')

In [101]:
covg_group = exposure.groupby(by='Covg_Code').mean()
covg_group['score'] = covg_group['Contract_Exposure'] + covg_group['Member_Exposure']
covg_group['score'] = np.log(covg_group['score'])

In [65]:
covg_dic = {}
for index, row in covg_group.iterrows():
    covg_dic[str(index)] = row['sum']
    
score = []
for index, row in tqdm(benefits.iterrows()):
    score.append(round(covg_dic[row['covg_code']],2))
benefits['score'] = score

In [150]:
claims = pd.read_csv('claim_aggs.csv')
claims['score'] = np.log(claims['Claim_Count'] + claims['Person_Count'])

In [161]:
claim_dic = {}
for index, row in claims.iterrows():
    claim_dic[row['Covg_Code']] = row['score']

In [202]:
score = []
for index, row in tqdm(benefits.iterrows()):
    if str(row['covg_code']) in claim_dic.keys():
        score.append(round(claim_dic[str(row['covg_code'])],2))
    else:
        score.append(0)
benefits['score_claims'] = score

423909it [01:24, 5023.30it/s]


In [591]:
def make_df(df):
    dic = {}
    for index, row in df.iterrows():
        if row['value_flag'] in ["Text","Numeric"]:
            question = row['answer'].split(':')[0]
            answer = row['answer'].split(':')[1:]
            first_score = row['score']
            second_score = row['score_claims']
            cov_code = row['covg_code']
            ans_tag = row['answer_tag']
            tag = row['type_of_tag']
            if row['top_50_flag'] == "Y":
                top_50 = 'top_50'
            else:
                top_50 = 'not_top_50'
            if question not in dic:
                dic[question] = [[answer],[answer],[cov_code],[first_score],[second_score],[top_50],[ans_tag],[tag]]
            else:
                sub_ = dic[question][0] + [answer]
                dic[question] = [[list(item) for item in set(tuple(row) for row in sub_)],dic[question][1] + [answer],list(set(dic[question][2] +[cov_code])),dic[question][3] + [first_score],dic[question][4] + [second_score],dic[question][5] + [top_50],list(set(dic[question][6] +[ans_tag])),list(set(dic[question][7] +[tag]))]
        if row['value_flag'] not in ["Text","Numeric"]:
            question = row['answer']
            answer = 'This is a non-text non-numeric answer'
            first_score = row['score']
            second_score = row['score_claims']
            cov_code = row['covg_code']
            ans_tag = row['answer_tag']
            tag = row['type_of_tag']
            if row['top_50_flag'] == "Y":
                top_50 = 'top_50'
            else:
                top_50 = 'not_top_50'
            if question not in dic:
                dic[question] = [answer,'none',[cov_code],[first_score],[second_score],[top_50],[ans_tag],[tag]]
            else:
                dic[question] = [answer,'none',list(set(dic[question][2]+[cov_code])),dic[question][3] + [first_score],dic[question][4] + [second_score],dic[question][5] + [top_50],list(set(dic[question][6] +[ans_tag])),list(set(dic[question][7] +[tag]))]
    return pd.DataFrame.from_dict(dic,orient='index')
    
        
    

In [592]:
df = make_df(benefits)

In [593]:
df.columns = ['unique_configs','all_configs','covg_codes','pop_score','claim_score','top50_status','ans_tag','type_tag']

In [594]:
num_config = []
for val1,val2 in zip(df['unique_configs'],df['all_configs']):
    if val2 == 'none':
        num_config.append(0)
    else:
        num_config.append(len(val1))
df['num_config'] = num_config

In [595]:
def mean(lst): 
    return sum(lst) / len(lst) 

In [596]:
opt_score = []
opt_claim = []
for val1,val2 in zip(df['pop_score'],df['claim_score']):
    opt_score.append(round(mean(val1),2))
    opt_claim.append(round(mean(val2),2))
df['mean_expo_score'] = opt_score
df['mean_claim_score'] = opt_claim

In [597]:
count50 = []
ratio50 = []
for val in df['top50_status']:
    top_50_count = val.count("top_50")
    top_50_ratio = round(val.count("top_50")/(val.count("top_50")+val.count("not_top_50")),2)
    count50.append(top_50_count)
    ratio50.append(top_50_ratio)
df['top_50_count'] = count50
df['top_50_ratio'] = ratio50

In [598]:
num_cov = []
for val in df['covg_codes']:
    num_cov.append(len(val))
df['num_cov_codes'] = num_cov

In [599]:
df.drop(['top50_status','pop_score','claim_score'],axis=1,inplace=True)

In [617]:
#df.to_csv('final_output.csv')